In [1]:
# -*- coding: utf-8 -*-

In [2]:
from bs4 import BeautifulSoup
import requests 
import re
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
url = 'https://www.studentski-servis.com/index.php?t=prostaDela&hp=true&page=1&perPage=3016&sort=1&workType=1&keyword='
       
url_data=requests.get(url).content

soup = BeautifulSoup(url_data,'html.parser')

In [98]:
# SEZNAMI:
jobs = soup.findAll('div',{'class':'jobItem'})
ime = soup.findAll('span', {'class':'title'})
placa = soup.findAll('span',{'class':'postavka'})
lokacija = soup.findAll('span',{'class':'lokacija'})
opis = soup.findAll('div',{'class':'jobContent col'})
opiss= soup.findAll('p')
sifre=[]
for i in range(len(jobs)):
    sifra = jobs[i].li.parent.li.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling
    sifre.append(sifra.text.strip('Šifra: ').strip(' |'))



dataset= [(s,i.text.strip('\n'),p.text.strip('\n').strip('\r\n\t\t\t\t\t').replace(',','.'),l.text.strip('\n'),op.text.replace('|\n','. ').replace('\n','').replace('\r','')) for s,i,p,l,op in zip( sifre,ime,placa,lokacija,opis)]


In [99]:
with open('estudent.csv','w+',encoding='utf-8') as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(['sifra','naziv','placa','lokacija','opis'])
    for tuple in dataset:
        if tuple[0] is csvfile.read():
            None
        else:
            writer.writerow(tuple)



In [100]:
data=pd.read_csv('estudent.csv')
pd.options.display.max_rows = 20
data['neto']=data['placa']

vzorec_neto = re.compile(r"(?P<neto>.*?) €\/h neto",flags=re.DOTALL)

vrednosti=[]
for i in range(len(data['neto'])):
    if re.findall(vzorec_neto,data['neto'][i]):
        vrednosti+=re.findall(vzorec_neto,data['neto'][i])
    else: vrednosti.append('')
vrednosti
data['neto']=vrednosti
data

,sifra,naziv,placa,lokacija,opis,neto
0,321253,KOMERCIALNA DELANABAVA,PO DOGOVORU...,NOVO MESTO,V hitro rastočem podjetju multinacionalnih str...,
1,321252,POMOČ PRI PRODAJI,4.56 €/h neto (5.40 €/h bruto),LJUBLJANA VIČ,Podjetje išče polnoletno osebo za pomoč pri pr...,4.56
2,321251,FIZIČNA DELAARHEOLOŠKA IZKOPAVANJA,5.00 €/h neto (5.92 €/h bruto),ŠKOFJA LOKA,"Lažja do srednje težka fizična dela na terenu,...",5.00
3,321250,BLAGAJNIK,5.07 €/h neto (6.00 €/h bruto),LJUBLJANA ŠIŠKA,Potrebujejo resne in odgovorne študente za del...,5.07
4,321249,MARKETING,6.00 €/h neto (7.10 €/h bruto),KOMENDA,Skrb in promoviranje spletnih strani podjetja....,6.00
...,...,...,...,...,...,...
966,275186,PROGRAMIRANJE,PO DOGOVORU/PROJEKTU...,LJUBLJANA,"Zaželeno poznavanje: PL/SQL, JAVASCRIPT, C,C++...",
967,270843,TRŽENJE,10.00 €/h neto (11.83 €/h bruto),DOLENJSKA,Promocija in trženje pri vnaprej dogovorjenih ...,10.00
968,270842,TRŽENJE,10.00 €/h neto (11.83 €/h bruto),ŠTAJERSKA,Promocija in trženje pri vnaprej dogovorjenih ...,10.00
969,270841,TRŽENJE,10.00 €/h neto (11.83 €/h bruto),LJUBLJANA Z OKOLICO,Promocija in trženje pri vnaprej dogovorjenih ...,10.00


In [101]:
df=data[['sifra','naziv','lokacija','neto']]
coredf=df[df.neto != ''] #izbrisali smo vse sluzbe ki nimajo podane neto plače
coredf

,sifra,naziv,lokacija,neto
1,321252,POMOČ PRI PRODAJI,LJUBLJANA VIČ,4.56
2,321251,FIZIČNA DELAARHEOLOŠKA IZKOPAVANJA,ŠKOFJA LOKA,5.00
3,321250,BLAGAJNIK,LJUBLJANA ŠIŠKA,5.07
4,321249,MARKETING,KOMENDA,6.00
5,321248,STREŽBA HRANE IN PIJAČE,DOLENJSKE TOPLICE,5.07
...,...,...,...,...
964,286805,FIZIČNA DELA,RADOMLJE,4.65
967,270843,TRŽENJE,DOLENJSKA,10.00
968,270842,TRŽENJE,ŠTAJERSKA,10.00
969,270841,TRŽENJE,LJUBLJANA Z OKOLICO,10.00


In [107]:
po_mestih=coredf.groupby('lokacija').size()
po_mestih


lokacija
ADLEŠIČI                 1
ALEJA LJUBLJANA-ŠIŠKA    1
ANAKRAN                  1
ANKARAN                  2
APAČE                    1
                        ..
ŠMARTNO PRI LITIJI       4
ŠTAJERSKA                1
ŽEJE PRI KOMENDI         1
ŽUSTERNA                 1
ŽUŽEMBERK                4
Length: 313, dtype: int64